In [1]:
from Trainer import NTrainer
from TheCar import Car
from time import sleep
from Carbase import CarBase
from Track import Track, CControlPoint
from utils.Vector2 import Vector2D
from PathPoint import PathP
from Sensors import Sensors
import numpy as np
import random
import math
from collections import deque

In [3]:
# трек
r = Track()
r.Load('images/track1')
r.Prepare()

#draw Car
from time import sleep
from ipycanvas import Canvas, hold_canvas

canvas = Canvas(width=1000, height=600)
display(canvas)

def draw_cars(car, n):
    with hold_canvas(canvas):
        # Clear the old animation step
        canvas.clear()
        for p in r.Points:  
            canvas.stroke_style = 'blue'
            canvas.stroke_line(p.LeftWall.From.X,p.LeftWall.From.Y,p.LeftWall.To.X,p.LeftWall.To.Y)
            canvas.stroke_line(p.RightWall.From.X,p.RightWall.From.Y,p.RightWall.To.X,p.RightWall.To.Y)
        canvas.fill_style = 'red'
        if car.Breaks:
            canvas.fill_style = 'orange'
        canvas.fill_rect(car.Pos.X, car.Pos.Y, 5, 5)
  
        canvas.fill_style = 'red'
        canvas.font = '14px serif'
        canvas.fill_text(str(car.Velocity), 0, 10, max_width=None)
        canvas.fill_text(n, 0, 25, max_width=None)
        canvas.fill_text(car.GetPassLength(), 0, 40, max_width=None)
        canvas

        canvas.sleep(10)
        
Tr = NTrainer()
Tr.Track = r
Tr.Population = 10

# Tr.GeneticNN.ActivationFunctionType = TanHFunction
dt = 0.1  # скорость симуляции
generationCount =10
for i in range(generationCount):
        Tr.BuildFirstPopulation()
        Tr.BuildNextGeneration()

        while (True):
            if (not Tr.IsAlive()):
                if (Tr.Generation >= generationCount):
                    break
                Tr.BuildNextGeneration()
            draw_cars(Tr.find_leader(), i)
            # если лидер прошёл более 10 000 метров, то сохранить его веса в файл
            if Tr.find_leader().TotalPassedLength >= 10000:
                Tr.GeneticNN.NNs[Tr.find_leader().Index].SaveWeights("images/net1.pt")
                break
            # update cars
            Tr.Update(dt)         
# если сеть не обучается, то просто запустите ещё раз!

Canvas(height=600, width=1000)

In [3]:
# проверка сети на других треках
nr = Track()
nr.Load('images/americas')    # другой трек
nr.Prepare()
Tr.Track = nr
Tr.BuildFirstPopulation("images/net.pt")    # ранее сохранённая сеть
generationCount = 10
dt = 0.3

canvas = Canvas(width=1000, height=600)
display(canvas)

for i in range(3000):
    with hold_canvas(canvas):
        canvas.clear()
        Tr.Update(dt)
        for p in nr.Points:  
            canvas.stroke_style = 'blue'
            canvas.stroke_line(p.LeftWall.From.X,p.LeftWall.From.Y,p.LeftWall.To.X,p.LeftWall.To.Y)
            canvas.stroke_line(p.RightWall.From.X,p.RightWall.From.Y,p.RightWall.To.X,p.RightWall.To.Y)
        canvas.fill_style = 'red'
        car = Tr.find_leader()
        if car.Breaks:
            canvas.fill_style = 'orange'
        canvas.fill_rect(car.Pos.X, car.Pos.Y, 5, 5)
  
        canvas.fill_style = 'red'
        canvas.font = '14px serif'
        canvas.fill_text(str(car.Velocity), 0, 10, max_width=None)
        canvas.fill_text(car.Index, 0, 25, max_width=None)
        canvas.fill_text(car.GetPassLength(), 0, 40, max_width=None)
        canvas
        
        #Tr.Update(dt)
        if (not Tr.IsAlive()):
            if (Tr.Generation >= generationCount):
                break

        canvas.sleep(10)

Canvas(height=600, width=1000)

KeyboardInterrupt: 